# HydroCell Quantum Tunneling Simulation

This notebook demonstrates the quantum mechanical tunneling effects in a hydrogen fuel cell model, visualizing how electrons traverse the proton exchange membrane barrier.

In [ ]:
import sys
sys.path.append("../src")

import numpy as np
import matplotlib.pyplot as plt
from src.hydrocell import FuelCell

# Create fuel cell with parameters optimized for observable tunneling
cell = FuelCell(
    T=298.15, 
    PH2=1.0, 
    PO2=1.0, 
    membrane_resistance=0.05,
    barrier_width=0.5,  # nm - thinner barrier shows more tunneling
    barrier_height=3.0,  # eV
    electron_energy=2.7   # eV - closer to barrier height
)

# Run simulation
result = cell.simulate(load_resistance=0.3)

# Print results
for k, v in result.items():
    print(f"{k.capitalize()}: {v:.6f}")

## Wavefunction Visualization

Below we visualize the electron wavefunction and the potential barrier. In quantum mechanics, electrons behave as waves that can penetrate barriers even when their energy is less than the barrier height - a phenomenon known as quantum tunneling.

The key elements in the visualization:
- **Probability density** ($|\psi|^2$): Represents the probability of finding the electron at each position
- **Potential barrier**: The energy barrier electrons must overcome
- **Electron energy level**: When below the barrier height, classical physics predicts no transmission, but quantum mechanics allows tunneling

In [ ]:
# Improved visualization with error handling
try:
    x, psi = cell.solve_schrodinger()
    
    # Convert to nanometers for display
    x_nm = x * 1e9
    
    # Calculate potential energy profile (in eV for display)
    V_profile = np.array([cell.potential_barrier(pos) / cell.e for pos in x])
    
    # Create figure with two subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
    
    # Plot probability density with proper normalization
    prob_density = np.abs(psi)**2
    if np.max(prob_density) > 0:  # Avoid division by zero
        prob_density = prob_density / np.max(prob_density)  # Normalize for visibility
    
    ax1.plot(x_nm, prob_density, 'b', linewidth=2, label='Probability density $|psi|^2$')
    ax1.fill_between(x_nm, 0, prob_density, alpha=0.3, color='blue')
    ax1.set_ylabel('Normalized probability density', fontsize=12)
    ax1.set_title('Electron Wavefunction', fontsize=14)
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # Plot potential barrier and electron energy level
    ax2.plot(x_nm, V_profile, 'r', linewidth=2, label='Potential barrier')
    ax2.axhline(y=cell.electron_energy, color='g', linestyle='--', 
               label=f'Electron energy ({cell.electron_energy} eV)')
    ax2.fill_between(x_nm, 0, V_profile, where=(V_profile > 0), 
                    alpha=0.3, color='red')
    ax2.set_xlabel('Position (nm)', fontsize=12)
    ax2.set_ylabel('Energy (eV)', fontsize=12)
    ax2.set_title('Potential Energy Profile', fontsize=14)
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print tunneling metrics
    tunneling_prob = cell.compute_tunneling_probability()
    print(f"Tunneling probability: {tunneling_prob:.6f}")
    print(f"Quantum factor: {cell.quantum_enhanced_current(1.0):.6f}")
    
except Exception as e:
    print(f"Error in visualization: {str(e)}")
    # Fallback to simple plot
    plt.figure(figsize=(10, 6))
    plt.title("Potential Barrier")
    barrier_x = np.linspace(-2*cell.barrier_width_m, 3*cell.barrier_width_m, 1000)
    barrier_y = [cell.potential_barrier(x)/cell.e for x in barrier_x]
    plt.plot(barrier_x*1e9, barrier_y)
    plt.axhline(y=cell.electron_energy, color='g', linestyle='--')
    plt.xlabel("Position (nm)")
    plt.ylabel("Energy (eV)")
    plt.show()

## Mathematical Foundation of Tunneling

The tunneling probability through a rectangular barrier is given by:

$$T \approx \exp\left(-2\kappa L\right)$$

where:
- $L$ is the barrier width
- $\kappa = \sqrt{\frac{2m(V_0-E)}{\hbar^2}}$ is the decay constant
- $V_0$ is the barrier height
- $E$ is the electron energy
- $m$ is the electron mass
- $\hbar$ is the reduced Planck constant

This explains why:
1. Thinner barriers (smaller $L$) have higher tunneling probabilities
2. Lower barriers (smaller $V_0$) are easier to tunnel through
3. Higher energy electrons (larger $E$) tunnel more readily

In [ ]:
# Demonstrate the effect of barrier parameters on tunneling probability
import pandas as pd

# Parameter ranges to test
widths = np.linspace(0.2, 1.0, 5)  # nm
heights = np.linspace(2.5, 4.0, 4)  # eV

# Create results table
results = []
for w in widths:
    for h in heights:
        test_cell = FuelCell(
            T=298.15, PH2=1.0, PO2=1.0, membrane_resistance=0.05,
            barrier_width=w, barrier_height=h, electron_energy=2.7
        )
        prob = test_cell.compute_tunneling_probability()
        results.append({'Width (nm)': w, 'Height (eV)': h, 'Tunneling Probability': prob})

# Display as DataFrame
df = pd.DataFrame(results)
pd.pivot_table(df, values='Tunneling Probability', 
               index='Width (nm)', columns='Height (eV)', 
               aggfunc='mean').style.format("{:.8f}")

In [ ]:
def solve_schrodinger(self):
    """Improved solver for the time-independent Schrödinger equation"""
    # Use logarithmically spaced points to better resolve the barrier region
    left_domain = np.linspace(-5*self.barrier_width_m, -0.1*self.barrier_width_m, 200)
    barrier_domain = np.linspace(-0.1*self.barrier_width_m, 1.1*self.barrier_width_m, 400)
    right_domain = np.linspace(1.1*self.barrier_width_m, 5*self.barrier_width_m, 200)
    x_range = np.concatenate([left_domain, barrier_domain, right_domain])
    x_range = np.sort(np.unique(x_range))  # Ensure no duplicate points
    
    # Wave vector k outside barrier
    k = np.sqrt(2 * self.m_e * self.electron_energy_J) / self.hbar
    
    # Handle cases where energy > barrier vs energy < barrier
    if self.electron_energy_J >= self.barrier_height_J:
        # For E > V, use the analytical solution directly since there's no tunneling
        # Just reflection/transmission at a potential step
        return self.analytical_above_barrier(x_range)
    else:
        # For E < V, use a combination of analytical and numerical approaches
        try:
            # Define the Schrödinger equation as a first-order system
            def schrodinger_system(x, y):
                psi, phi = y
                V = np.zeros_like(x)
                for i, xi in enumerate(x):
                    V[i] = self.potential_barrier(xi)
                
                # Avoid division by zero by adding a small epsilon
                epsilon = 1e-10
                dpsi_dx = phi
                dphi_dx = 2 * self.m_e / (self.hbar**2 + epsilon) * (V - self.electron_energy_J) * psi
                return np.vstack((dpsi_dx, dphi_dx))
            
            # Improved boundary conditions based on physical understanding
            def bc(ya, yb):
                # Left side: incident + reflected wave
                # Right side: transmitted wave only
                k_outside = np.sqrt(2 * self.m_e * self.electron_energy_J) / self.hbar
                
                # Enforce continuity and derivative matching
                return np.array([
                    # At left boundary: psi is normalized
                    ya[0] - 1.0,
                    # At right boundary: derivative matches transmitted wave
                    yb[1] - 1j * k_outside * yb[0]
                ])
            
            # More robust initial guess
            y_guess = np.zeros((2, len(x_range)))
            
            # For x < 0
            mask_left = x_range < 0
            y_guess[0, mask_left] = np.exp(1j * k * x_range[mask_left]) + 0.5 * np.exp(-1j * k * x_range[mask_left])
            y_guess[1, mask_left] = 1j * k * np.exp(1j * k * x_range[mask_left]) - 0.5 * 1j * k * np.exp(-1j * k * x_range[mask_left])
            
            # For 0 <= x <= barrier_width
            kappa = np.sqrt(2 * self.m_e * abs(self.barrier_height_J - self.electron_energy_J)) / self.hbar
            mask_barrier = (x_range >= 0) & (x_range <= self.barrier_width_m)
            y_guess[0, mask_barrier] = 0.5 * np.exp(-kappa * x_range[mask_barrier])
            y_guess[1, mask_barrier] = -0.5 * kappa * np.exp(-kappa * x_range[mask_barrier])
            
            # For x > barrier_width
            mask_right = x_range > self.barrier_width_m
            y_guess[0, mask_right] = 0.25 * np.exp(1j * k * (x_range[mask_right] - self.barrier_width_m))
            y_guess[1, mask_right] = 0.25 * 1j * k * np.exp(1j * k * (x_range[mask_right] - self.barrier_width_m))
            
            # Use complex solver for accurate phases
            sol = solve_bvp(schrodinger_system, bc, x_range, y_guess.real, 
                           max_nodes=10000, tol=1e-3, verbose=0)
            
            if not sol.success:
                # Fall back to analytical approximation if numerical solution fails
                return self.analytical_tunneling(x_range)
            
            return sol.x, sol.y[0]
            
        except Exception as e:
            print(f"Numerical solver failed: {str(e)}")
            # Fallback to analytical approximation
            return self.analytical_tunneling(x_range)
    
def analytical_tunneling(self, x_range):
    """Analytical approximation for tunneling wavefunction"""
    # Wave vectors
    k_outside = np.sqrt(2 * self.m_e * self.electron_energy_J) / self.hbar
    kappa = np.sqrt(2 * self.m_e * (self.barrier_height_J - self.electron_energy_J)) / self.hbar
    
    # Transmission and reflection coefficients (approximate)
    T = 4 * k_outside * kappa / ((k_outside + kappa)**2) * np.exp(-kappa * self.barrier_width_m)
    R = 1 - T  # Conservation of probability
    
    # Construct wavefunction
    psi = np.zeros_like(x_range, dtype=complex)
    for i, x in enumerate(x_range):
        if x < 0:
            # Incident + reflected wave
            psi[i] = np.exp(1j * k_outside * x) + np.sqrt(R) * np.exp(-1j * k_outside * x)
        elif 0 <= x <= self.barrier_width_m:
            # Evanescent wave in barrier
            psi[i] = (1 + np.sqrt(R)) * np.exp(-kappa * x)
        else:
            # Transmitted wave
            psi[i] = np.sqrt(T) * np.exp(1j * k_outside * (x - self.barrier_width_m))
    
    return x_range, psi

def analytical_above_barrier(self, x_range):
    """Analytical solution for electron energy above barrier height"""
    # Wave vectors
    k_outside = np.sqrt(2 * self.m_e * self.electron_energy_J) / self.hbar
    k_inside = np.sqrt(2 * self.m_e * (self.electron_energy_J - self.barrier_height_J)) / self.hbar
    
    # Reflection and transmission at each interface
    r1 = (k_outside - k_inside) / (k_outside + k_inside)
    t1 = 2 * k_outside / (k_outside + k_inside)
    
    r2 = (k_inside - k_outside) / (k_inside + k_outside)
    t2 = 2 * k_inside / (k_inside + k_outside)
    
    # Phase accumulated across barrier
    phase = k_inside * self.barrier_width_m
    
    # Total transmission coefficient
    T = abs(t1 * t2 * np.exp(1j * phase) / (1 + r1 * r2 * np.exp(2j * phase)))**2
    R = 1 - T
    
    # Construct wavefunction
    psi = np.zeros_like(x_range, dtype=complex)
    for i, x in enumerate(x_range):
        if x < 0:
            # Incident + reflected wave
            psi[i] = np.exp(1j * k_outside * x) + np.sqrt(R) * np.exp(-1j * k_outside * x)
        elif 0 <= x <= self.barrier_width_m:
            # Oscillatory wave in barrier
            # Approximate behavior - real solution would need to match at boundaries
            psi[i] = t1 * (np.exp(1j * k_inside * x) + r2 * np.exp(1j * k_inside * (2 * self.barrier_width_m - x)))
        else:
            # Transmitted wave
            psi[i] = np.sqrt(T) * np.exp(1j * k_outside * x)
    
    return x_range, psi